## <center> RecSys. Home Assignment 1</center>

Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке вам нужно потренироваться в этом и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении своих рекомендательных моделей. 

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл. 
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл. 
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла. 
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **24 октября 23:59**. 

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла [Name_Surname]_HW1.ipynb.  

Удачи!

In [1]:
from typing import NamedTuple, Union
import tests
import tests2
import torch

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )

# Precision

$$P@k = \frac{1}{k} \sum_{i=1}^k [y_{i} = 1]$$

In [4]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    precision_at_k = torch.mean(target_sorted_by_output[:, :topk])
    return precision_at_k

In [6]:
tests.run_precision(precision)

# Recall

$$R@k = \frac{\sum_{i=1}^k [y_{i} = 1]}{\sum_{i=1}^N [y_{i} = 1]}$$

In [7]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    if torch.sum(target_sorted_by_output) == 0:
        recall_at_k = 0.
    else:
        recall_at_k = torch.mean(torch.sum(target_sorted_by_output[:, :topk], dim=-1) /  torch.sum(target_sorted_by_output, dim=-1))
    return recall_at_k

In [9]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP@k = \sum_{i=1}^{k} \frac{y_{i}}{\sum_{j=1}^{k} \cdot y_{j}} p@i$$
$$MNAP@k = \frac{1}{U} \sum_{u \in U} \frac{1}{min(k, n_u)} AP@k_{u},$$

где $n_u$ - количество айтемов с интеракциями у пользователя $u$ в тестовый период, <br>
$U$ - количество пользователей. 

In [11]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    res = 0.
    if normalized:
        min_ = torch.min(
            torch.full_like(torch.sum(target_sorted_by_output, dim=-1), topk),
            torch.sum(target_sorted_by_output, dim=-1)
        )
        res = torch.mean(torch.nan_to_num(1 / min_ * torch.sum((target_sorted_by_output * \
                                (torch.cumsum(target_sorted_by_output, dim=-1) / \
                                torch.tensor(range(1, target_sorted_by_output.shape[1] + 1))))[:, : topk], dim=-1)))
    else:
        res = torch.mean(torch.nan_to_num(torch.sum((target_sorted_by_output * (target_sorted_by_output.cumsum(dim=-1) / \
                                    torch.tensor(range(1, target_sorted_by_output.shape[1] + 1))))[:, : topk], dim=-1) / \
                                torch.sum(target_sorted_by_output[:, : topk], dim=-1)))           
    return res

In [15]:
tests.run_map(mnap)
tests.run_mnap(mnap)

AssertionError: Inputs:{
  "output": [
    [
      9.0,
      5.0,
      3.0,
      0.0,
      7.0,
      4.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      8.0,
      2.0,
      0.0,
      10.0
    ],
    [
      0.0,
      0.0,
      1.0,
      5.0,
      9.0,
      3.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      4.0,
      0.0,
      0.0,
      10.0,
      7.0,
      0.0,
      2.0,
      8.0,
      6.0
    ],
    [
      0.0,
      1.0,
      4.0,
      8.0,
      6.0,
      5.0,
      3.0,
      7.0,
      10.0,
      0.0,
      9.0,
      0.0,
      0.0,
      2.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0
    ],
    [
      7.0,
      8.0,
      0.0,
      0.0,
      1.0,
      0.0,
      4.0,
      0.0,
      10.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      9.0,
      2.0,
      3.0,
      5.0,
      0.0
    ]
  ],
  "target": [
    [
     

Я не менял исходную функцию сортировки, но у меня сортирует, не так как в тестах

In [12]:
output =  torch.Tensor(
                [
                    [9, 5, 3, 0, 7, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1, 8, 2, 0, 10],
                    [0, 0, 1, 5, 9, 3, 0, 0, 0, 0, 0, 4, 0, 0, 10, 7, 0, 2, 8, 6],
                    [0, 1, 4, 8, 6, 5, 3, 7, 10, 0, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0],
                    [7, 8, 0, 0, 1, 0, 4, 0, 10, 0, 0, 6, 0, 0, 0, 9, 2, 3, 5, 0],
                ]
            )
target = torch.Tensor(
                [
                    [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                    [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
                    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
                    [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
                ]
)

topk = 100

In [13]:
prepare_target(output, target)

tensor([[0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
         0., 0.],
        [0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
         1., 0.],
        [0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
         1., 0.]])

# Normalized Dicsounted Cumulative Gain


$$ NDCG @k = \frac{DCG@k}{IDCG@k},$$ где 
$$DCG@k = \sum_{i=1}^{k} \frac{2^{y_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG@k = \sum_{i=1}^{k} \frac{1}{log_2 (i + 1)}$$

In [16]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    ideal_target = prepare_target(target, target)
    dcg_at_k = torch.sum(dcg(target_sorted_by_output[:, :topk]), dim=-1)
    idcg_at_k = torch.sum(dcg(ideal_target[:, :topk]), dim=-1)
    ndcg_at_k = torch.mean(torch.nan_to_num(dcg_at_k / idcg_at_k))
    return ndcg_at_k

In [17]:
tests.run_ndcg(ndcg)

AssertionError: Inputs:{
  "output": [
    [
      9.0,
      5.0,
      3.0,
      0.0,
      7.0,
      4.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      8.0,
      2.0,
      0.0,
      10.0
    ],
    [
      0.0,
      0.0,
      1.0,
      5.0,
      9.0,
      3.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      4.0,
      0.0,
      0.0,
      10.0,
      7.0,
      0.0,
      2.0,
      8.0,
      6.0
    ],
    [
      0.0,
      1.0,
      4.0,
      8.0,
      6.0,
      5.0,
      3.0,
      7.0,
      10.0,
      0.0,
      9.0,
      0.0,
      0.0,
      2.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0
    ],
    [
      7.0,
      8.0,
      0.0,
      0.0,
      1.0,
      0.0,
      4.0,
      0.0,
      10.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      9.0,
      2.0,
      3.0,
      5.0,
      0.0
    ]
  ],
  "target": [
    [
     

Погрешность округления